# Assignment 2
Bradley Thompson - CS 510 Large Language Models PDX Winter 2024

## Experimental Setting 1
To start, I simply tried to set up my test bed for model performance. I missed the class where we disussed breaking down the model output
by getting the log probability of the result tokens, and while it makes sense in theory, I don't know how to do it in practice. So, I instead
decided to come up with a simple approach for checking on the generated output labels, by simply checking for substring presence and throwing out samples where a category can't be found. This is sub-par becuase it doesn't account for cases where multiple labels are selected in rambling output, and because it doesn't account for malformed output which might resemble a category option closely. I tried to remediate this at least slightly be limiting the number of new tokens generated, so that multiple labels were unlikely to be produced.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODELS = (
    "bigscience/bloom-560m",
    "bigscience/bloom-1b1",
    "bigscience/bloom-1b7",
    "bigscience/bloomz-560m",
    "bigscience/bloomz-1b1",
    "bigscience/bloomz-1b7",
)

model_name = MODELS[0]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [15]:
from datasets import load_dataset

dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english")
target_dataset = dataset["train"]

# Check out what the data looks like:
target_dataset[2]

{'text': '"Frank Gaffrey\\u002c Cliff May\\u002c Steve Emerson: Brilliant. \\""""Looming Threats: Iran\\u002c Hezbollah Hamas\\"""" is the best #cufidc session I\\u2019ve had thus far." ',
 'label': 2}

In [80]:
import math

POSITIVE="positive"
NEUTRAL="neutral"
NEGATIVE="negative"


# Note: we have 2 options so we can be case insensitive
LABELS = "negative Negative neutral Neutral positive Positive "
ID_TO_LABEL = {
    0: NEGATIVE,
    1: NEUTRAL,
    2: POSITIVE,
}

def get_id_by_labels_index(labels_index: int) -> int:
    """
    Find the target label id, same labels used in `tweet_sentiment_multilinqual` dataset.
    For a label (index) `n`, `floor(n/2)` gives our target label id in `ID_TO_LABEL`.
    """
    return math.floor(labels_index / 2)


target_label_tokens = tokenizer("positive Positive negative Negative neutral Neutral")["input_ids"]
print("Gathering vocab indices for target tokens:", tokenizer.decode(target_label_tokens))
target_label_tokens

Gathering vocab indices for target tokens: positive Positive negative Negative neutral Neutral


[96675, 139904, 23381, 149414, 40979, 76420, 4343]

In [95]:
from transformers import GenerationConfig
import torch as t

config = {
    "min_new_tokens": 1,
    "max_new_tokens": 1,
    # "use_cache": False,
    # "do_sample": True,
    # "top_k": 2,
}

HYPOTHESIS = "This text has a positive sentiment, true or false:"

def classify(premise: str) -> int:
    """
    Use model to generate output logits on target premise/hypothesis pair.
    Result based on probability of entailment for the given hypothesis.
    https://joeddav.github.io/blog/2020/05/29/ZSL.html#Classification-as-Natural-Language-Inference
    :param premise: some input text string to be classified based on `LABELS`.
    :returns: classification via index of target label in `LABELS`; use `get_labels_by_labels_index` for final label.
    """
    inputs = tokenizer.encode(premise, HYPOTHESIS, return_tensors="pt")
    gen_config: GenerationConfig = GenerationConfig.from_dict(config)
    # Get prediction scores across vocab for the only token generated b/c of gen config and normalize w/ softmax
    output = model.generate(inputs, gen_config, return_dict_in_generate=True, output_scores=True)["scores"][0]
    vocab_probs = output.softmax(dim=1)
    # Get probabilities of our target labels by index in our vocab
    labels_log_probs = t.index_select(vocab_probs, 1, t.tensor(target_label_tokens))[0]
    # Get highest log prob label
    labels_index = t.argmax(labels_log_probs).item()
    return get_id_by_labels_index(labels_index)

sample = target_dataset[0]["text"]
print(f"Input text: {sample}\nClassification: {ID_TO_LABEL[classify(sample)]}")

Input text: okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry) 
Classification: negative


In [10]:
import evaluate

recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

print(recall_metric.compute(references=[1, 1, 1, 0, 0], predictions=[1, 1, 0, 0, 0]))
print(precision_metric.compute(references=[1, 1, 1, 0, 0], predictions=[1, 1, 0, 0, 0]))
print(f1_metric.compute(references=[1, 1, 1, 0, 0], predictions=[1, 1, 0, 0, 0]))

{'recall': 0.6666666666666666}
{'precision': 1.0}
{'f1': 0.8}
